In [1]:
from network import Network
from layer_wta import LayerWTA
from layer_lsh import LayerLSH
from layer import Layer

import numpy as np
from keras.utils import to_categorical
from keras.metrics import CategoricalAccuracy

import pickle

# import dataset
from keras.datasets import mnist
# load dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

hidden_units = 256
learning_rate = 0.01

function_num = 5
table_num = 6

top_k = 1-(1-(1/2**function_num))**table_num

epochs = 25

2023-06-20 08:31:12.995164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 08:31:13.676948: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
neural_network_wta = Network()
neural_network_wta.addLayer(LayerWTA(input_size, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(LayerWTA(hidden_units, hidden_units, top_k))
neural_network_wta.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_wta, epoch_time_wta] = neural_network_wta.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs)
y_hat = neural_network_wta.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

2023-06-20 08:31:19.321457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Epoch  1:   3% |▎         | 1854/60000, Accuracy: 0.257;

In [ ]:
file_name = '../data/neural_network_wta.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(neural_network_wta, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "../data/neural_network_wta.pkl"


In [ ]:
neural_network_lsh = Network()
neural_network_lsh.addLayer(LayerLSH(input_size, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(LayerLSH(hidden_units, hidden_units, function_num=function_num, table_num=table_num))
neural_network_lsh.addLayer(Layer(hidden_units, num_labels))

[epoch_accuracy_lsh, epoch_time_lsh] = neural_network_lsh.fit(x_train, y_train, learning_rate=learning_rate, epochs = epochs)
y_hat = neural_network_lsh.predict(x_test)

metrics = CategoricalAccuracy()
metrics.update_state(y_test, y_hat)
print(metrics.result().numpy())

Epoch  1:   0% |          | 236/60000, Accuracy: 0.093;


KeyboardInterrupt: 

In [ ]:
file_name = '../data/neural_network_lsh.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(neural_network_lsh, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "../data/neural_network_lsh.pkl"
